# Initialization

## Importing relevant modules

In [2]:
import pandas as pd
from drg_connect import Snowflake
import qgrid 
from datetime import timedelta, datetime
import math

import warnings
warnings.filterwarnings('ignore')

## Connecting to snowflake

In [3]:
##defining parameters of snowflake
snow = Snowflake(role = 'RWD_ANALYTICS_RW',database='SANDBOX_ANALYTICS',schema = 'SANDBOX')
engine = snow.engine

%reload_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

diagnosis codes:

('Q611','Q612', 'Q613', '75314', '75313', '75314')

NDC codes

('5914808707', '5914808728', '5914808807', '5914808828', '5914808907', '5914808928', '5914802050', '5914802150')

In [6]:
%%read_sql

select *

from RWD_DB.RWD.ICD_GROUPER
where 
    level_4_long_description_icd10 ilike '%polycystic%'
    and level_4_long_description_icd10 ilike '%kidney%'
    --and level_4_short_description_icd10 ilike '%disease%'

Query started at 04:21:09 PM GMT Daylight Time; Query executed in 0.04 m

,id,level_1,level_1_description,level_2,level_2_description,level_3,level_3_description,level_4,icd9_mapped_codes,icd9_description,level_4_short_description_icd10,level_4_long_description_icd10,create_ts,update_ts
0,28033,Q00 - Q999,Congenital malformations deformations and chro...,Q60-Q64,Congenital malformations of the urinary system,Q61-Q619,Cystic kidney disease,Q611,No map,No map,Polycystic kidney infantile type,Polycystic kidney infantile type,2017-11-02,2017-11-02
1,28035,Q00 - Q999,Congenital malformations deformations and chro...,Q60-Q64,Congenital malformations of the urinary system,Q61-Q619,Cystic kidney disease,Q6119,75314,Polycystic kidney autosomal recessive,Other polycystic kidney infantile type,Other polycystic kidney infantile type,2017-11-02,2017-11-02
2,28036,Q00 - Q999,Congenital malformations deformations and chro...,Q60-Q64,Congenital malformations of the urinary system,Q61-Q619,Cystic kidney disease,Q612,75313,Polycystic kidney autosomal dominant,Polycystic kidney adult type,Polycystic kidney adult type,2017-11-02,2017-11-02
3,28037,Q00 - Q999,Congenital malformations deformations and chro...,Q60-Q64,Congenital malformations of the urinary system,Q61-Q619,Cystic kidney disease,Q613,75312,Polycystic kidney unspecified type,Polycystic kidney unspecified,Polycystic kidney unspecified,2017-11-02,2017-11-02
4,93591,Z00 - Z9989,Factors influencing health status and contact ...,Z77-Z99,Persons with potential health hazards related ...,Z82-Z828,Fam hx of certain disabil & chr dis (leading t...,Z8271,V1861,Family history of polycystic kidney,Family history of polycystic kidney,Family history of polycystic kidney,2017-11-02,2017-11-02


In [8]:
df = pd.read_excel("ref.xlsx")

In [9]:
df

,cat1,value
0,rx,5914808707
1,rx,5914808728
2,rx,5914808807
3,rx,5914808828
4,rx,5914808907
5,rx,5914808928
6,rx,5914802050
7,rx,5914802150
8,dx,Q611
9,dx,Q612


In [10]:
snow.upload_dataframe(df, "nk_adpkd_ref")

Upload successful!


# Dx Extract

In [12]:
%%read_sql

create or replace table nk_adpkd_dx as

select
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
    claim_number,
    diagnosis,
    diagnosis_sequence,
    year_of_service
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS
where
    diagnosis in (select value from nk_adpkd_ref where cat1 = 'dx')

Query started at 04:31:25 PM GMT Daylight Time; Query executed in 10.74 m

,status
0,Table NK_ADPKD_DX successfully created.


## Getting index date

In [18]:
%%read_sql

create or replace table nk_adpkd_index as

select
    patient_id,
    min(year_of_service) as index_date
from
    nk_adpkd_dx
group by 1

Query started at 04:59:13 PM GMT Daylight Time; Query executed in 0.05 m

,status
0,Table NK_ADPKD_INDEX successfully created.


# Pharmacy extract

In [31]:
%%read_sql

create or replace table nk_adpkd_rx as

select 
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
    claim_number,
    product_or_service_id as ndc,
    date_of_service
from 
    RWD_DB.RWD.RAVEN_PHARMACY
where
    product_or_service_id in ('46602002105','75839016905','46602002005','59148002050','59148002150')

Query started at 05:17:34 PM GMT Daylight TimeInitiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
; Query executed in 14.51 m

,status
0,Table NK_ADPKD_RX successfully created.


In [33]:
a = snow.select("select count(patient_id) from nk_adpkd_index")
b = snow.select("select count(distinct patient_id) from nk_adpkd_rx")
c = snow.select("select count(distinct patient_id) from nk_adpkd_rx where patient_id in (select patient_id from nk_adpkd_index)")

lst = ['pkd',
       'tovalptan',
       'pks_and_tovalptan']

pd.concat([a,b,c], keys=lst, axis=1)

,pkd,tovalptan,pks_and_tovalptan
,COUNT(PATIENT_ID),COUNT(DISTINCT PATIENT_ID),COUNT(DISTINCT PATIENT_ID)
0,131195,6710,67


In [ ]:
in all of raven, there are about 131K pts matching on those diagnosis codes, there are 6.7k matching on the drug codes for tovalptan (independent of diagnosis codes) and the overlap between those two is 67 patients